# Reinforcement Learning in Finite MDPs

In [0]:
!git clone https://github.com/rlgammazero/mvarl_hands_on.git > /dev/null 2>&1

## MDPs

In [0]:
import sys
sys.path.insert(0, './mvarl_hands_on/utils')
import numpy as np
from scipy.special import softmax # for SARSA
import matplotlib.pyplot as plt
import json
import math
from cliffwalk import CliffWalk

Setting up the environment

In [0]:
env = CliffWalk(proba_succ=0.98)
#from test_env import ToyEnv1
####################################################################################
# You probably want to test smaller enviroments before
#env = ToyEnv1(gamma=0.99)
####################################################################################

# Useful attributes
print("Set of states:", env.states)
print("Set of actions:", env.actions)
print("Number of states: ", env.Ns)
print("Number of actions: ", env.Na)
print("P has shape: ", env.P.shape)  # P[s, a, s'] = env.P[s, a, s']
print("discount factor: ", env.gamma)
print("")

# Usefult methods
state = env.reset() # get initial state
print("initial state: ", state)
print("reward at (s=1, a=3,s'=2): ", env.reward_func(1,3,2))
print("")

# A random policy
policy = np.random.randint(env.Na, size = (env.Ns,))
print("random policy = ", policy)

# Interacting with the environment
print("(s, a, s', r):")
for time in range(4):
    action = policy[state]
    next_state, reward, done, info = env.step(action)
    print(state, action, next_state, reward)
    if done:
        break
    state = next_state
print("")

## Question 1: Value iteration
1. Write a function applying the optimal Bellman operator on a provided Q function: $Q_1 = LQ_0, \; Q_0\in \mathbb{R}^{S\times A}$
2. Write a function implementing Value Iteration (VI) with $\infty$-norm stopping condition (reuse function implemented in 1)
3. Evaluate the convergence of your estimate, i.e., plot the value $\|V_n - V^\star\|_{\infty}$

In [0]:
# --------------
# Point 1
# --------------
def bellman_operator(Q0, Ns, Na, R, P,gamma):
    ''' 
    Q0: initial matrix
    Ns: number of states
    Na: number of actions
    R: reward matrix
    P: policy matrix
    Greedy_policy: optimal bellman operator
    '''
    max_Q=np.max(Q0,axis=1)
    R_2d= np.sum(P*R, axis=2)
    Q1=R_2d+gamma*np.dot(P,max_Q)
    greedy_policy=np.argmax(Q1, axis=1)    
    return Q1, greedy_policy

In [0]:
# --------------
# Point 2
# --------------
def value_iteration(Q0, env, epsilon=1e-5):
    Q_history=[Q0]
    condition = True 
    while condition :
        Ns,Na,R,P= env.Ns, env.Na,env.R, env.P
        Q, greedy_policy=bellman_operator(Q_history[-1], Ns, Na, R, P,env.gamma)
        condition=np.linalg.norm(Q-Q_history[-1],ord=np.inf)> epsilon
        Q_history.append(Q)
    # TODO (feel free to change the return argument
    return Q, greedy_policy, Q_history

In [0]:
# --------------
# Point 3
# --------------
with open("./mvarl_hands_on/data/Q_opts.json", "r") as fp:
    Qopts = json.load(fp)
Qstar = Qopts["{}_{}".format(type(env).__name__,env.gamma)]

Q0=np.zeros(np.shape(Qstar))
 
Q, greedy_policy, Q_history = value_iteration(Q0, env)

norm_values=[np.linalg.norm(Q-Qstar,np.inf) for Q in Q_history]

max_values=[np.max(Q-Qstar) for Q in Q_history]

# TODO

plt.plot(norm_values)
plt.xlabel('Iteration')
plt.ylabel('Error')
plt.title("Q-learning: Convergence of Q ($infinity$ norm)")
plt.show()

plt.plot(max_values)
plt.xlabel('Iteration')
plt.ylabel('max Error')
plt.title("Q-learning: Convergence of Q (max error)")
plt.show()


In [0]:
state = env.reset()
env.render()
for i in range(50):
    action = greedy_policy[state]
    state, reward, done, _ = env.step(action)
    env.render()

## Question 2: Q learning
Q learning is a model-free algorithm for estimating the optimal Q-function online.
It is an off-policy algorithm since the samples are collected with a policy that is (potentially) not the one associated to the estimated Q-function.

1. Implement Q learning with $\epsilon$-greedy exploration.
  - Plot the error in Q-functions over iterations
  - Plot the cumulative sum of rewards

In [0]:
# ---------------------------
# Q-Learning
# ---------------------------
# suggested interface
# you can change it!
class QLearning:
    """
    Q learning with epsilon-greedy exploration
    """
    def __init__(self,env,Q0,state):
        self.env=env
        self.Q=Q0
        self.state=state
        self.alpha=None
        self.count_states=1
        pass
    
    def sample_action(self, state, greedy):
        # best_action = random_action*(1-epsilon)+ greedy_action*epsilon
        #####
        epsilon=0.3
        action1=np.max(greedy[state])
        action2= np.random.choice(self.env.actions)
        theta = np.random.binomial(1,1-epsilon)
        best_action = action1 * theta + action2 * (1-theta)
        return best_action
        
        
    def update(self, state, action, next_state, reward):
        delta=reward+ np.max(self.Q[next_state][:])-self.Q[state][action]
        self.count_states+=1
        self.alpha=1./(self.count_states)
        self.Q = self.Q + self.alpha * delta 
        self.state = np.argmax(self.Q,axis=0)
        

In [0]:


# --------------
# Point 1
# --------------
# Number of Q learning steps
max_steps = int(1e5)  
# max_steps = 10

Q0 = np.zeros((env.Ns, env.Na))
# Use the previous code to verify the correctness of q learning
Q_opt, pi_opt,_ = value_iteration(Q0, env, epsilon=1e-8)


# main algorithmic loop
norm_values = []
t = 0
s0=0
ql=QLearning(env,Q0,s0)

while t < max_steps:
    action =ql.sample_action(state, greedy_policy) 
    observation, reward, done, info = env.step(action)
    norm_values.append(np.abs(ql.Q - Q_opt).mean())
    ### update the Q function
    next_state = observation
    ql.update(state, action, next_state, reward)
    # todo do something
    t = t + 1

print(env.render())
print("optimal policy: ", pi_opt)
greedy_policy = np.argmax(ql.Q, axis=1)
print("best policy:", greedy_policy)


plt.plot(norm_values)
plt.xlabel('Iteration')
plt.ylabel('Error')
plt.title("Q-learning: Convergence of Q")

# how confident are you in the performance of the algorithm? maybe a single run is not enough